In [1]:
import psycopg2
import psycopg2.extras

The galactic pipeline works as follows.  On designated "reference" images— right now I just have one per filter— it finds all sources using Eddie Schlafly's crowdsource algorithm, and saves them to the galsources table in the database.

Then, for every other image (call them "search" images), it will run through the same reduction procedure.  It finds sources but does _not_ save them, instead just using them for astrometric and (sort of) photometric calibration.  After that, it reads all the sources from the reference image (of the relevant filter) from the database, and uses the "force fit" version of crowdsource to force there to be sources at all the same positions on the search image as they were on the ref image.  Because I noticed ~10% variations across the chip in the comparison between source and ref, I do a correction to the source magnitudes so that in bulk they match the ref magnitudes; that correction is a 4th-order chebyshev polynomnial in magnituide difference as a function of x and y across the image.  These corrected magnitudes for all of the sources that had >0 flux on the search image are saved to the database.  What this means is that the zeropoint saved in the database for the image is only an approximation.  We're more interested in relative photometry than absolute photometry, so I've calibrated the search image to the reference image.  The real photometric system is defined by the reference image, but then the zeropoint of the reference image should give you absolute calibration to within 10% or 20%.

Finally, the pipeline looks at all of the sources in the search image for magnitude differences from the ref image.  Right now it finds a __lot__.  I do a few things.  First, when saving the catalog of the search image, I enforce an "uncertainty floor" of 0.01 magnitudes.  I don't believe that crowdsource can be trusted to better than this, even though it often quotes sub-0.01 magnitude uncertainties.  (It's probably not to better than 2%, in fact.)  There's also issues of how damn difficult it is to adequately subtract the sky in crowded fields.  I stopped using crowdsources' sky subtraction, and instead went with the Bijaoui algorithm (done in a 20x10 tiles to which a smooth background is fit), as I was getting better results with that.  Second, when I do the correction fit, I look at the χ²/ν value, and expand the uncertainties in the search image by a factor that will make the χ²/ν value go to 1.  (So far, typically, this has been a factor of a few.)  Finally, I do a 15σ cut on the magnitude difference, which sounds high, but the number of variables was overwhelming without that.  We can still tweak this pipeline if we don't like this.

We still get a lot of variables.  However, many of them will be "one-offs"; that is, the result of an image artifact or something like that.  We'll want to focus more on the objects that are detected multiple times as differnt from the ref image.  One flaw in the pipeline right now is that an artifact in the _ref_ image will cause something to be detected multiple times.  I want to think about how to deal with that.  (Possiblities include: using mutiple ref images (all photometrically calibrated to one of them), stacked ref image with outlier rejection.)

All of the tables for the galactic pipeline start with "gal"; the relevant ones are:
* galexposures — each full 62-chip exposure downloaded from the NOIRLab data archive
* galimages — one record for each chip of each exposure; galexposure_id points to the galexposures table
* galsources — this is the __GIANT__ table (as in 10⁷ rows per exposure).  Sources detected on images.  galimage_id points to the galimages table.
* galvarobjs — variable objects detectected where the galsources entry for an image was significantly different from the galsources entry for that same object for the relevant reference image.  (The galimages field "reference" tells you which image was used as a reference for a given search image

I'm hoping that the q3c extension will make searching the ginormous galsources table by ra/dec relatively quick.  This does mean I have to regularly run a very slow "cluster" process to make the q3c index effective, which seems to lock up the galsources table while I'm doing it.  We may decide that saving all the sources from every image is too much for the database, but it would be really nice if we could keep that, as it means that when a variable object is detected, all of the photometry that we've succesfully grabbed for that object is already right there in the database.

Right now, I'm only working in "development" mode, which means that a smallish number of images have been loaded into the decat_dev database, and they may get deleted or munged about as I keep working.  However, if you want to look at the sorts of things that are coming up, this is the database to look at.  __TO LOOK AT THE PRODUCTION DATABASE__: replace 'decat_dev' with 'decat' after dbname below.

In [2]:
db = psycopg2.connect("dbname='decat_dev' user='mgraham' password='PutYourPasswordHere' host='decatdb.lbl.gov'")

In [37]:
db.rollback()

In [38]:
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

Here's a query that will get counts of how many times a given object was detected as variable (i.e. different from the ref).  The "refsource_id" will be the same for the same objects.  (This is _within a filter_.  The same star in g and and r will have different refsource ids.  Cross-filter identification will require searches by RA/DEC, and then there will be blending issues if the photometry on the g and r refs didn't deblend the objects the same way!  The galaxy has too many stars.)

In [3]:
query = ( "SELECT num,COUNT(num) as count "
          "FROM ( SELECT COUNT(v.id) as num "
                 "FROM galvarobjs v "
                 "GROUP BY refsource_id ) subq "
          "GROUP BY num "
          "ORDER BY num DESC" )
cursor.execute( query )
print( "Repeated Detections  Number of Objects" )
for row in cursor.fetchall():
    print( f'    {row["num"]:8d}               {row["count"]:8d}' )

Repeated Detections  Number of Objects
           9                     63
           8                    401
           7                    496
           6                   1850
           5                   2520
           4                   1407
           3                   1930
           2                   3437
           1                  27786


Let's look at some of the ones that had 9 detections.

In [11]:
query = ( "SELECT num,refsource_id,mag "
          "FROM ( SELECT COUNT(v.id) as num,v.refsource_id,r.mag "
                 "FROM galvarobjs v "
                 "INNER JOIN galsources r ON v.refsource_id=r.id " 
                 "GROUP BY refsource_id,r.mag ) subq "
          "WHERE num=9 "
          "LIMIT 10" )
cursor.execute(query)
print( f"{'#dets':>10s} {'refid':>10s} {'mag':>5s}" )
for row in cursor.fetchall():
    print( f'{row["num"]:10d} {row["refsource_id"]:10d} {row["mag"]:5.2f}' )


     #dets      refid   mag
         9  284560863 17.89
         9  284771217 15.98
         9  284780308 16.49
         9  284917017 18.78
         9  285155891 15.90
         9  285183148 16.15
         9  285224121 16.18
         9  285306251 16.07
         9  286513214 17.65
         9  286682205 20.37


Let's pull out the detections of 284560863. As with all queries that hits the galsources table, this can take some time.

In [23]:
query = ( "SELECT e.mjd,e.filter,e.is_reference,s.mag,s.magerr "
          "FROM galvarobjs v "
          "INNER JOIN galsources s ON v.galsource_id=s.id "
          "INNER JOIN galimages i ON s.galimage_id=i.id "
          "INNER JOIN galexposures e ON i.galexposure_id=e.id "
          "WHERE v.refsource_id=284560863 "
          "ORDER BY e.mjd" )
cursor.execute( query )
for row in cursor.fetchall():
    print( f'{row["mjd"]:12.2f}   {row["filter"]:6s}   {row["mag"]:6.2f}   {row["magerr"]:6.2f}   {row["is_reference"]}' )

    59295.39   r         17.04     0.01   None
    59295.39   r         17.01     0.01   None
    59295.39   r         17.02     0.01   None
    59295.40   r         17.00     0.01   None
    59295.40   r         17.04     0.01   None
    59295.41   r         17.01     0.01   None
    59313.38   r         17.09     0.01   False
    59322.37   r         18.46     0.02   False
    59343.42   r         17.07     0.05   False


Let's try to pull out the lightcurve of that source.  Start by getting the RA and DEC from the galsource table entry from the reference image:

In [29]:
query = ( "SELECT s.ra,s.dec FROM galsources s WHERE s.id=284560863" )
cursor.execute( query )
for row in cursor.fetchall():
    print( f'RA = {row["ra"]}    DEC = {row["dec"]}' )

RA = 271.486119431597    DEC = -28.9101692547778


A q3c_radial_query searches for everything within a given radius of a given RA/DEC.  The radius is in degrees. I will search for sources that are 0.3" away from the RA and DEC of this ref source:

In [34]:
query = ( "SELECT e.mjd,e.filter,e.is_reference,s.ra,s.dec,s.mag,s.magerr "
          "FROM galsources s "
          "INNER JOIN galimages i ON s.galimage_id=i.id "
          "INNER JOIN galexposures e ON i.galexposure_id=e.id "
          "WHERE q3c_radial_query(s.ra,s.dec,271.486119431597,-28.9101692547778,0.3/3600.) "
          "ORDER BY e.filter,e.mjd" )
cursor.execute( query )
for row in cursor.fetchall():
    print( f'{row["mjd"]:12.2f}   {row["filter"]:6s}   {row["mag"]:6.2f}   {row["magerr"]:6.2f}  {row["is_reference"]}' )

    59295.39   g         17.84     0.01  None
    59295.39   g         17.85     0.01  None
    59295.40   g         17.85     0.01  None
    59295.40   g         17.85     0.01  None
    59295.40   g         17.86     0.01  None
    59297.40   g         18.87     0.00  True
    59331.33   g         17.81     0.03  None
    59295.39   i         16.83     0.01  None
    59295.39   i         16.81     0.01  None
    59295.39   i         16.79     0.01  None
    59295.40   i         16.81     0.01  None
    59295.40   i         16.80     0.01  None
    59295.41   i         16.81     0.01  None
    59297.40   i         17.56     0.00  True
    59295.39   r         17.04     0.01  None
    59295.39   r         17.01     0.01  None
    59295.39   r         17.02     0.01  None
    59295.40   r         17.00     0.01  None
    59295.40   r         17.04     0.01  None
    59295.41   r         17.01     0.01  None
    59297.40   r         17.89     0.00  True
    59313.38   r         17.09    

The last column, "is reference", is a tag in the exposures column that this exposure is used as a reference for finding variable objects.  This doesn't guarantee it was the reference for *this* object!  However, at the moment, I only have a single reference for each filter, so it does guarantee that.

The thing I notice is that on mdj 5297, in each filter the star is systematically different from most of the other observations... and 5297 is the reference!  Something was going on with that star that day.  I would say it was an image artifact, but for the fact that it's an outlier in all four filters.  HOWEVER, I've only got something like 3 days in the databse, so this could be a real varaible that just didnt' vary much on one day.  This will be more meaningful when there are more separate days in the database.  (That may never happen in the dev server, but when we go production, it will.)

(NOTE that if you run this query, you may well get different results, as I've probably added things to the database.)

Let's now do the same thing on another source.

In [41]:
query = ( "SELECT s.ra,s.dec FROM galsources s WHERE s.id=284917017" )
cursor.execute( query )
row = cursor.fetchone()
ra = row['ra']
dec = row['dec']
query = ( "SELECT e.mjd,e.filter,e.is_reference,s.ra,s.dec,s.mag,s.magerr "
          "FROM galsources s "
          "INNER JOIN galimages i ON s.galimage_id=i.id "
          "INNER JOIN galexposures e ON i.galexposure_id=e.id "
          "WHERE q3c_radial_query(s.ra,s.dec,%s,%s,0.5/3600.) "
          "ORDER BY e.filter,e.mjd" )
cursor.execute( query, ( ra, dec ) )
for row in cursor.fetchall():
    print( f'{row["mjd"]:12.2f}   {row["filter"]:6s}   {row["mag"]:6.2f}   {row["magerr"]:6.2f}  {row["is_reference"]}' )

    59295.39   g         19.05     0.02  None
    59295.39   g         19.01     0.02  None
    59295.40   g         18.96     0.02  None
    59295.40   g         18.94     0.02  None
    59295.40   g         18.91     0.02  None
    59297.40   g         19.84     0.01  True
    59331.33   g         18.54     0.02  None
    59295.39   i         17.94     0.01  None
    59295.39   i         17.91     0.02  None
    59295.39   i         17.88     0.02  None
    59295.40   i         17.84     0.02  None
    59295.40   i         17.84     0.02  None
    59295.41   i         17.80     0.02  None
    59297.40   i         18.48     0.01  True
    59295.39   r         18.18     0.01  None
    59295.39   r         18.18     0.02  None
    59295.39   r         18.11     0.01  None
    59295.40   r         18.06     0.02  None
    59295.40   r         18.03     0.02  None
    59295.41   r         18.01     0.02  None
    59297.40   r         18.78     0.01  True
    59313.38   r         17.71    